In [1]:
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, AutoModel, get_scheduler
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error
import pandas as pd
from tqdm import tqdm


In [2]:
model_name = "bert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_df = pd.read_csv("/content/trac2_CONVT_train.csv", sep=',', engine='python', on_bad_lines='skip', escapechar='\\')
dev_df = pd.read_csv("/content/trac2_CONVT_dev.csv", sep=',', engine='python', on_bad_lines='skip', escapechar='\\')
test_df = pd.read_csv("/content/trac2_CONVT_test.csv", sep=',', engine='python', on_bad_lines='skip', escapechar='\\')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
def tokenize_texts(texts):
  return tokenizer(
      list(texts),
      truncation=True,
      padding="max_length",
      max_length=128,
      return_tensors="pt"
  )
train_tokens = tokenize_texts(train_df["text"])
dev_tokens = tokenize_texts(dev_df["text"])
test_tokens = tokenize_texts(test_df["text"])
for col in ["Emotion", "EmotionalPolarity"]:
  train_df[col] = train_df[col].astype(int) - train_df[col].min()
  dev_df[col] = dev_df[col].astype(int) - dev_df[col].min()
train_emotion = torch.tensor(train_df["Emotion"].values, dtype=torch.float)
train_polarity = torch.tensor(train_df["EmotionalPolarity"].values, dtype=torch.long)
train_empathy = torch.tensor(train_df["Empathy"].values, dtype=torch.float)
dev_emotion = torch.tensor(dev_df["Emotion"].values, dtype=torch.float)
dev_emotional_polarity = torch.tensor(dev_df["EmotionalPolarity"].values, dtype=torch.long)
dev_empathy = torch.tensor(dev_df["Emotion"].values, dtype=torch.float)




/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
class MultiTaskModel(nn.Module):
  def __init__(self, model_name, polarities):
    super().__init__()
    self.bert = AutoModel.from_pretrained(model_name)
    hidden_size = self.bert.config.hidden_size
    dropout = self.bert.config.hidden_dropout_prob
    self.drop = nn.Dropout(dropout)
    self.emotion_classifier = nn.Linear(hidden_size, 1)
    self.polarity_classifier = nn.Linear(hidden_size, polarities)
    self.empathy = nn.Linear(hidden_size, 1)
  def forward(self, input_id, attention_mask):
    outputs = self.bert(input_ids=input_id, attention_mask=attention_mask)
    cls_output = self.drop(outputs.last_hidden_state[:, 0, :])
    emotion = self.emotion_classifier(cls_output).squeeze(-1)
    polarity = self.polarity_classifier(cls_output)
    empathy = self.empathy(cls_output).squeeze(-1)
    return emotion, polarity, empathy


In [5]:
train_dataset = TensorDataset(
    train_tokens["input_ids"], train_tokens["attention_mask"], train_emotion, train_polarity, train_empathy
)
dev_dataset = TensorDataset(
    dev_tokens["input_ids"], dev_tokens["attention_mask"], dev_emotion, dev_emotional_polarity, dev_empathy
)

In [6]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=16)

In [7]:
num_polar_labels = len(sorted(train_df["EmotionalPolarity"].unique()))
model = MultiTaskModel(model_name, num_polar_labels).to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 5
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

In [8]:
def train_epochs(model, dataloader, optimizer, scheduler):
  model.train()
  total_loss = 0.0
  emotion_loss = nn.MSELoss()
  polarity_loss = nn.CrossEntropyLoss()
  empathy_loss = nn.MSELoss()
  for batch in tqdm(dataloader, desc="Training", leave=False):
    input_ids, attention_mask, emo_labl, emo_pol_lab, emp_lbl = [x.to(device) for x in batch]
    optimizer.zero_grad()
    emo_logits, emo_pol_logits, emp_logits = model(input_ids, attention_mask)
    loss = emotion_loss(emo_logits, emo_labl) + polarity_loss(emo_pol_logits, emo_pol_lab) + empathy_loss(emp_logits, emp_lbl)
    loss.backward()
    optimizer.step()
    scheduler.step()
    total_loss += loss.item()
  return total_loss / len(dataloader)


In [9]:
def evaluate(model, dataloader):
  model.eval()
  emo_preds, emo_pol_preds, emp_preds = [], [], []
  emo_lbls, emo_pol_lbls, emp_lbls = [], [], []
  with torch.no_grad():
    for batch in tqdm(dataloader, desc="Evaluating", leave=False):
      input_ids, attention_mask, emo_labl, emo_pol_lab, emp_lbl = [x.to(device) for x in batch]
      emo_logits, emo_pol_logits, emp_logits = model(input_ids, attention_mask)

      emo_preds.extend(emo_logits.cpu().numpy().tolist())
      emo_pol_preds.extend(torch.argmax(emo_pol_logits, dim=1).cpu().numpy().tolist())
      emp_preds.extend(emp_logits.cpu().numpy().tolist())

      emo_lbls.extend(emo_labl.cpu().numpy().tolist())
      emo_pol_lbls.extend(emo_pol_lab.cpu().numpy().tolist())
      emp_lbls.extend(emp_lbl.cpu().numpy().tolist())
  return {
      "Emotion_MAE": mean_absolute_error(emo_lbls, emo_preds),
      "Emotional_Polarity_Accuracy": accuracy_score(emo_pol_lbls, emo_pol_preds),
      "Empathy_MAE": mean_absolute_error(emp_lbls, emp_preds),
      "Polarity_F1": f1_score(emo_pol_lbls, emo_pol_preds, average="weighted")

  }

In [11]:
for epoch in range(num_epochs):
  train_loss = train_epochs(model, train_loader, optimizer, lr_scheduler)
  metrics = evaluate(model, dev_loader)
  print(f"\nEpoch {epoch+1}/{num_epochs}")
  print(f"Train Loss: {train_loss:.4f}")
  print(f"Emotion MAE: {metrics['Emotion_MAE']:.4f}")
  print(f"Empathy MAE: {metrics['Empathy_MAE']:.4f}")
  print(f"Emotional Polarity Accuracy: {metrics['Emotional_Polarity_Accuracy']:.4f}, F1: {metrics['Polarity_F1']:.4f}")
  print("-" * 50)



Epoch 1/5
Train Loss: 1.9602
Emotion MAE: 0.4976
Empathy MAE: 0.5512
Emotional Polarity Accuracy: 0.4071, F1: 0.3526
--------------------------------------------------



Epoch 2/5
Train Loss: 1.4755
Emotion MAE: 0.5016
Empathy MAE: 0.5726
Emotional Polarity Accuracy: 0.3859, F1: 0.3369
--------------------------------------------------



Epoch 3/5
Train Loss: 1.3187
Emotion MAE: 0.4844
Empathy MAE: 0.5663
Emotional Polarity Accuracy: 0.3990, F1: 0.3521
--------------------------------------------------



Epoch 4/5
Train Loss: 1.2122
Emotion MAE: 0.5015
Empathy MAE: 0.5937
Emotional Polarity Accuracy: 0.3960, F1: 0.3515
--------------------------------------------------



Epoch 5/5
Train Loss: 1.1368
Emotion MAE: 0.5020
Empathy MAE: 0.6054
Emotional Polarity Accuracy: 0.3808, F1: 0.3358
--------------------------------------------------


In [12]:
model.eval()
emo_preds, emo_pol_preds, emp_preds = [], [], []
test_ids = test_df["id"].tolist()
input_ids, mask = test_tokens["input_ids"], test_tokens["attention_mask"]
with torch.no_grad():
  for i in range(0, len(input_ids), 16):
    batch_ids = input_ids[i:i+16].to(device)
    batch_mask = mask[i:i+16].to(device)
    emo_logits, emo_pol_logits, emp_logits = model(batch_ids, batch_mask)
    emo_preds.extend(emo_logits.cpu().numpy().tolist())
    emo_pol_preds.extend(torch.argmax(emo_pol_logits, dim=1).cpu().numpy().tolist())
    emp_preds.extend(emp_logits.cpu().numpy().tolist())
pred_df = pd.DataFrame({
    "id": test_ids,
    "Emotion": emo_preds,
    "EmotionalPolarity": emo_pol_preds,
    "Empathy": emp_preds
})
pred_df.to_csv("predictions_bert.csv", index=False)


In [ ]:
# Q5: Extract dev predictions for 5 conversations
conv_ids = [68, 72, 74, 80, 85]

model.eval()
print("="*70)
print("BERT DEV PREDICTIONS FOR Q5")
print("="*70)

for conv_id in conv_ids:
    conv_data = dev_df[dev_df['conversation_id'] == conv_id].head(5)
    conv_indices = conv_data.index.tolist()
    
    # Get tokenized inputs for these specific rows
    conv_input_ids = dev_tokens["input_ids"][conv_indices]
    conv_attention_mask = dev_tokens["attention_mask"][conv_indices]
    
    with torch.no_grad():
        emo_logits, pol_logits, emp_logits = model(conv_input_ids.to(device), 
                                                     conv_attention_mask.to(device))
        preds_e = emo_logits.cpu().numpy()
        preds_p = pol_logits.argmax(dim=1).cpu().numpy()
        preds_emp = emp_logits.cpu().numpy()
    
    print(f"\nCONV {conv_id}:")
    for i, (idx, row) in enumerate(conv_data.iterrows()):
        print(f"  T{int(row['turn_id'])}: E_pred={preds_e[i]:.2f} (true={row['Emotion']:.1f}), "
              f"P_pred={int(preds_p[i])} (true={int(row['EmotionalPolarity'])}), "
              f"Emp_pred={preds_emp[i]:.2f} (true={row['Empathy']:.1f})")

print("\n" + "="*70)




======================================================================
BERT DEV PREDICTIONS FOR Q5
======================================================================

CONV 68:
  T1: E_pred=0.96 (true=1.0), P_pred=1 (true=1), Emp_pred=0.93 (true=1.0)
  T2: E_pred=1.31 (true=2.0), P_pred=1 (true=1), Emp_pred=0.77 (true=2.0)
  T3: E_pred=2.25 (true=2.0), P_pred=0 (true=1), Emp_pred=1.33 (true=2.0)
  T4: E_pred=2.04 (true=2.0), P_pred=1 (true=1), Emp_pred=2.62 (true=3.0)
  T5: E_pred=3.69 (true=3.0), P_pred=2 (true=2), Emp_pred=3.34 (true=4.0)

CONV 72:
  T1: E_pred=0.70 (true=1.0), P_pred=1 (true=1), Emp_pred=1.02 (true=1.0)
  T2: E_pred=0.62 (true=1.0), P_pred=1 (true=1), Emp_pred=0.95 (true=1.0)
  T3: E_pred=0.95 (true=1.0), P_pred=1 (true=1), Emp_pred=1.01 (true=1.0)
  T4: E_pred=0.99 (true=1.0), P_pred=1 (true=1), Emp_pred=1.11 (true=1.0)
  T5: E_pred=2.39 (true=2.0), P_pred=1 (true=1), Emp_pred=1.69 (true=1.0)

CONV 74:
  T1: E_pred=0.82 (true=1.0), P_pred=1 (true=1), Emp_pred=0.88 (true=1.0)
  T2: E_pred=1.22 (true=1.0), P_pred=1 (true=1), Emp_pred=1.15 (true=1.0)
  T3: E_pred=3.77 (true=3.0), P_pred=2 (true=2), Emp_pred=4.28 (true=4.0)
  T4: E_pred=3.41 (true=3.0), P_pred=2 (true=2), Emp_pred=3.89 (true=4.0)
  T5: E_pred=2.26 (true=2.0), P_pred=1 (true=1), Emp_pred=2.12 (true=4.0)

CONV 80:
  T1: E_pred=0.54 (true=0.0), P_pred=1 (true=1), Emp_pred=0.96 (true=1.0)
  T2: E_pred=0.87 (true=0.0), P_pred=1 (true=1), Emp_pred=0.87 (true=1.0)
  T3: E_pred=3.84 (true=4.0), P_pred=2 (true=2), Emp_pred=2.95 (true=4.0)
  T4: E_pred=2.05 (true=2.0), P_pred=1 (true=1), Emp_pred=2.55 (true=2.0)
  T5: E_pred=1.67 (true=2.0), P_pred=1 (true=1), Emp_pred=1.53 (true=1.0)

CONV 85:
  T1: E_pred=1.49 (true=3.0), P_pred=0 (true=0), Emp_pred=0.80 (true=1.0)
  T2: E_pred=1.66 (true=1.0), P_pred=0 (true=0), Emp_pred=1.22 (true=1.0)
  T3: E_pred=2.49 (true=3.0), P_pred=2 (true=2), Emp_pred=2.63 (true=4.0)
  T4: E_pred=1.92 (true=2.0), P_pred=2 (true=2), Emp_pred=2.10 (true=3.0)
  T5: E_pred=2.75 (true=3.0), P_pred=2 (true=2), Emp_pred=2.82 (true=2.0)

======================================================================